In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install pymorphy2

In [2]:
import numpy as np
import pandas as pd
import re
from wordcloud import WordCloud

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords_ru = stopwords.words("russian")
from nltk.stem import WordNetLemmatizer
import pymorphy2
from matplotlib import pyplot
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
#import transformers
#import torch
#import tensorflow as tf
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
#from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
#from transformers import BertTokenizer, BertModel, BertConfig

import transformers
from transformers import DistilBertTokenizer
from transformers import DistilBertModel, DistilBertConfig
from transformers import TFDistilBertForSequenceClassification
import tensorflow as tf
import json
import gc

In [3]:
train = pd.read_csv('../input/scan-classification-challange/df_train.csv')
train.sample(3)

In [4]:
# Удалим дубликаты

train.drop_duplicates(subset={'text'}, inplace=True) 
train.shape

In [5]:
# # Названия категорий переводим в числовой формат и записываем в отдельный столбец

train['encoded_cat'] = train['class'].astype('category').cat.codes
train.sample(5)

In [6]:
test = pd.read_csv('../input/scan-classification-challange/df_test.csv', index_col=0)
test.sample(3)

In [7]:
# Посмотрим на состав имеющихся стоп-слов
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords_ru = stopwords.words("russian")
print(stopwords.words("russian"))

In [8]:
# Теперь выведем все слова длиной менее 3-х символов и дополним список стоп-слов
# пробовал искать среди стопслов равных 4-м, но там получаем слова 'газа', 'прав', 'дела', 'Рост'
# как мне кажется данные слова, могут быть полезны для score
stopworlds_new = set()
mas_stop = set()
for words in train['text']:
    for i in words.split():
        if len(i) <= 3:
            mas_stop.add(i)
stopworlds_new = set(stopwords_ru).union(mas_stop)

In [9]:
# Создадим функцию по очистке данных. Будем переводить слова в нижний регистр, 
# удалять стоп слова, удалять числа и раздичные знаки которые не несут смысловой нагрузки. 
# Все слова преобразуем к их первоначалоной форме (Лемматизация)

morph = pymorphy2.MorphAnalyzer()
patterns = "[A-Z|a-z|0-9!#$%&'()*+,./:“″;”<=>?@[\]^_`{|}~—\"\-•–«»]+"
#stops = set(stopwords.words("russian"))
def clean(text):
    text = text.lower()
    text = re.sub(patterns, ' ', text)
    tokens = []
    for token in text.split():
        if token and token not in stopworlds_new:
            token = token.strip()
            token = morph.normal_forms(token)[0]  # Лемматизация
            #token = stemmer.stem(token) # Стеммизация
            tokens.append(token)
    return ' '.join(tokens)

In [10]:
# Применим функцию очистки к train и test

train['clean'] = train['text'].apply(lambda x: clean(x))
test['clean'] = test['text'].apply(lambda x: clean(x))
train[['clean', 'text']]

## FastText

In [ ]:
ft_train, ft_test = train_test_split(train, random_state=42, test_size=0.2, stratify = train['class'])

In [ ]:
def to_ft_label(s):
    return '__label__'+s.replace(',','_').replace(' ','_').replace('-','_')

labels_dict = {}
for g in train['class']:
    labels_dict[to_ft_label(g)] = g

In [ ]:
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import remove_stopwords
train.iloc[:, 0] = train.iloc[:, 0].apply(lambda x: ' '.join(simple_preprocess(x)))
test.iloc[:, 1] = test.iloc[:, 1].apply(lambda x: ' '.join(simple_preprocess(x)))

In [ ]:
col = ['class', 'text']

# train
train_for_ft = ft_train[col]
train_for_ft['class']=[to_ft_label(s) for s in train_for_ft['class']]

# test
test_for_ft = ft_test[col]
test_for_ft['class']=[to_ft_label(s) for s in test_for_ft['class']]

In [ ]:
train_for_ft.to_csv('train_for_ft.csv', index=False, sep=' ', header=False, escapechar=" ")
test_for_ft.to_csv('test_for_ft.csv', index=False, sep=' ', header=False, escapechar=" ")

In [ ]:
import fasttext
model = fasttext.train_supervised('train_for_ft.csv', lr = 0.9)

In [ ]:
model.test('test_for_ft.csv')

In [ ]:
def predict(test):
    return labels_dict[ model.predict(test['text'], k=1)[0][0] ]
test['predictions'] = test.apply(predict,axis=1)

test.head()

In [ ]:
submission = pd.DataFrame({'id':range(0, len(test)),
                           'class':test['predictions'].values},
                          columns=['id', 'class'])
submission.to_csv('submission1.csv', index=False)
submission.head()

In [ ]:
#sub_predict = model.predict([text_sub_sequences, X_sub])
#sample_submission['class'] = sub_predict_nn2[:,0]
#sample_submission.to_csv('submission.csv', index=False)

## Model sklearn
Logistic Regression

In [11]:
# Выделим X, y. X - это будет, наш обработанный текст, y -  наш класс

y = train.encoded_cat.values
X = train.drop(['encoded_cat', 'text'], axis=1)


In [12]:
## Разделим все данные на train test в соотношении 80/20

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify=y)

Необходимо преобразовать тест в токены, сделаем несколькими способами и посмотрим на результат

In [13]:
## CountVectorizer преобразует текст в матрицу количества токенов

from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer()

X_train_review_bow = vect.fit_transform(X_train['clean'])
X_test_review_bow = vect.transform(X_test['clean'])
X_sub_rev_bow = vect.transform(test['clean'])
#X_train_review_bow = vect.fit_transform(X_train)
#X_test_review_bow = vect.transform(X_test)

print('X_train_review_bow shape: ', X_train_review_bow.shape)
print('X_test_review_bow shape: ', X_test_review_bow.shape)
print('X_sub_review_bow shape: ', X_sub_rev_bow.shape)

In [14]:
## Tf-Idf преобразует текст в матрицу функций TF-IDF (частота обратная частоте документа)

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

X_train_review_tfidf = vectorizer.fit_transform(X_train['clean'])
X_test_review_tfidf = vectorizer.transform(X_test['clean'])
X_sub_review_tfidf = vectorizer.transform(test['clean'])

print('X_train_review_tfidf shape: ', X_train_review_tfidf.shape)
print('X_test_review_tfidf shape: ', X_test_review_tfidf.shape)
print('X_sub_review_tfidf shape: ', X_sub_review_tfidf.shape)

In [15]:
# Построим модель логистической регресии, в качестве метрики будем выводить accuracy и F1

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

clf = LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='l2', random_state=42)
clf.fit(X_train_review_tfidf, y_train)

#y_pred = clf.predict(X_test_review_tfidf)
y_pred = clf.predict(X_test_review_tfidf)
print('Test Accuracy: ', accuracy_score(y_test, y_pred))
print('Test F1: ', f1_score(y_test, y_pred, average='weighted'))

In [16]:
clf = LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='l2', random_state=42)
clf.fit(X_train_review_bow, y_train)

y_pred = clf.predict(X_test_review_bow)
print('Test Accuracy: ', accuracy_score(y_test, y_pred))
print('Test F1: ', f1_score(y_test, y_pred, average='weighted'))

Попробуем другие модели sklearn

In [17]:
# C-классификация опорных векторов

from sklearn.svm import SVC
svclassifier = SVC(kernel='linear')
svclassifier.fit(X_train_review_bow, y_train)
y_pred_svc = svclassifier.predict(X_test_review_bow)
print('Test Accuracy: ', accuracy_score(y_test, y_pred_svc))
print('Test F1: ', f1_score(y_test, y_pred, average='weighted'))

In [18]:
# Классификатор, реализующий голосование k ближайших соседей

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 10)
knn.fit(X_train_review_bow, y_train)
y_pred = knn.predict(X_test_review_bow)

print('Test Accuracy: ', accuracy_score(y_test, y_pred))
print('Test F1: ', f1_score(y_test, y_pred, average='weighted'))

Гипер параметры

In [20]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

"""model = LogisticRegression(multi_class='multinomial', penalty='l2', random_state=42)
#penalty = ['l1','l2']
solver = ['newton-cg', 'sag', 'saga', 'lbfgs']
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
alpha = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
grid = dict(alpha=alpha)
c_values = [100, 10, 1.0]
#param_grid = {'C':[1, 10]}
param_grid = dict(C=c_values, solver=solver)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, verbose=10, cv=cv, scoring='accuracy', error_score=0)
grid_search.fit(X_train_review_bow, y_train)
#print("Best: %f using %s" % (grid_search.best_params_))"""

In [21]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(multi_class='multinomial', solver='newton-cg', penalty='l2', C = 10, random_state=42)
clf.fit(X_train_review_bow, y_train)

y_pred = clf.predict(X_test_review_bow)
print('Test Accuracy: ', accuracy_score(y_test, y_pred))
print('Test F1: ', f1_score(y_test, y_pred, average='weighted'))

In [22]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(multi_class='multinomial', solver='saga', penalty='l2', C = 100, random_state=42)
clf.fit(X_train_review_bow, y_train)

y_pred = clf.predict(X_test_review_bow)
print('Test Accuracy: ', accuracy_score(y_test, y_pred))
print('Test F1: ', f1_score(y_test, y_pred, average='weighted'))

Стекинг

In [ ]:
from sklearn.ensemble import StackingClassifier
#from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

#data, target = load_breast_cancer(return_X_y=True)

estimators = [('lr', LogisticRegression()), ('svc', SVC(kernel='linear'))]
modelClf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())

#X_train, X_valid, y_train, y_valid = train_test_split(data, target, test_size=0.3, random_state=12)

modelClf.fit(X_train_review_bow, y_train)
y_pred = clf.predict(X_test_review_bow)
print('Test Accuracy: ', accuracy_score(y_test, y_pred))
print('Test F1: ', f1_score(y_test, y_pred, average='weighted'))
#print(modelClf.score(X_valid, y_valid))

## Bert TenzorFlow

In [ ]:
!pip install transformers
!pip install pytorch
!pip install pytorch-transformers

In [ ]:
import transformers
from transformers import DistilBertTokenizer
from transformers import DistilBertModel, DistilBertConfig
from transformers import TFDistilBertForSequenceClassification
import tensorflow as tf
import json
import gc

In [ ]:
# Токенизируем текст 

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
X_train_encodings = tokenizer(X_train['clean'].to_list(), truncation=True, padding=True)
test_encodings = tokenizer(test['text'].to_list(), truncation=True, padding=True)

X_test_encodings = tokenizer(X_test['clean'].to_list(), truncation=True, padding=True)

In [ ]:
# Создает dataset с тонекизированными данными

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(X_train_encodings),
    y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((dict(X_test_encodings), 
                                    list(y_test))) 
val_class = np.zeros(len(test))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    val_class))

In [ ]:
class F1_Score(tf.keras.metrics.Metric):

    def __init__(self, name='f1_score', **kwargs):
        super().__init__(name=name, **kwargs)
        self.f1 = self.add_weight(name='f1', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        p = Precision(thresholds=0.5)(y_true, y_pred)
        r = Recall(thresholds=0.5)(y_true, y_pred)
        self.f1 = 2 * ((p * r) / (p + r + 1e-6))

    def result(self):
        return self.f1

    def reset_states(self):
        self.f1.assign(0)

In [ ]:
from keras import backend as K

def recall_m(y_train, y_test):
    true_positives = K.sum(K.round(K.clip(y_train * y_test, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_train, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_train, y_test):
    true_positives = K.sum(K.round(K.clip(y_train * y_test, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_test, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_train, y_test):
    precision = precision_m(y_train, y_test)
    recall = recall_m(y_train, y_test)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=50)
losss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5) # 2e-5
model.compile(optimizer=optimizer, loss=model.hf_compute_loss(), metrics=['accuracy', f1_m]) # loss=model.compute_loss

In [ ]:
model.fit(train_dataset.shuffle(1000).batch(16), 
          epochs=10,
          validation_data=test_dataset.shuffle(1000).batch(16))

## Distill-Bert fine-tuning - Huggingface and Pytorch

In [ ]:
#!pip install datasets
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from datasets import Dataset
from sklearn.model_selection import train_test_split

In [ ]:
id2label = {str(i): label for i, label in enumerate(train["class"].unique().tolist())}
label2id = {v: k for k, v in id2label.items()}
print(label2id)

In [ ]:
# Токенизатор DistilBERT будет принимать обучающие данные только в том случае, если у него есть labels столбец, 
# поэтому мы добавим еще один столбец с нашими сопоставленными метками.

train = (train.assign(labels=train["class"].map(label2id)))
train.head()

In [ ]:
# Создаем Dataset и разбиваем на test и train

from datasets import Dataset
from sklearn.model_selection import train_test_split

dataset = Dataset.from_pandas(train).train_test_split(train_size=0.8, seed=123)
print(dataset)

In [ ]:
# Затем мы создаем Datasetиз фрейма данных и разбиваем его на a testи trainустанавливаем.

dataset = dataset.class_encode_column("labels")

In [ ]:
# загружаем трансформатор AutoTokenizer для DistilBERT

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
#  Составляем список столбцов, которые нужно удалить из набора данных при токенизации

cols_to_remove = [col for col in dataset["train"].column_names if col != "labels"]
print(cols_to_remove)

In [ ]:
# После завершения токенизации мы подготавливаем набор данных для передачи в модель, устанавливая его формат "torch"

def tokenize(batch):
    tokenized_batch = tokenizer(batch['text'], padding=True, truncation=True, max_length=128)
    tokenized_batch['labels'] = [int(label) for label in batch["labels"]]
    return tokenized_batch

dataset_enc = dataset.map(tokenize, batched=True, remove_columns=cols_to_remove, num_proc=4)
dataset_enc.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

print(dataset_enc["train"].column_names)

In [ ]:
# Создаем экземпляр a DataLoader для каждого разделения набора данных, чтобы передать его модели

from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

train_dataloader = DataLoader(
    dataset_enc["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    dataset_enc["test"], batch_size=8, collate_fn=data_collator
)

In [ ]:
# Загружаем модель

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import AutoModelForSequenceClassification

# Load model from checkpoint
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", 
                                                           num_labels=50)

In [ ]:
# Определяем гтперпараметры, оптимизатор, скорость, планировщик

from transformers import AdamW
from transformers import get_scheduler

learning_rate = 5e-5
num_epochs = 3

optimizer = AdamW(model.parameters(), lr=learning_rate)

num_training_batches = len(train_dataloader)
num_training_steps = num_epochs * num_training_batches
lr_scheduler = get_scheduler(
    "linear",                   
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [ ]:
# Перемещаем модель на GPU

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

model.to(device)

In [ ]:
# Запускаем обучение модели

from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

# Train the model with PyTorch training loop
model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

In [ ]:
# Оценка модели

from datasets import load_metric

# Load metric
metric = load_metric("glue", "mrpc")

# Iteratively evaluate the model and compute metrics
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

# Get model accuracy and F1 score
metric.compute(predictions=predictions, references=batch["labels"], average="weighted")
#metric.compute(predictions=predictions, references=batch["labels"],average="weighted")

## Bert PyTorch

In [ ]:
# Создадим словарь из названия класса и его кодированног числа
labels = train.set_index('class').to_dict()['encoded_cat']

In [ ]:
from transformers import BertTokenizer

#tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased'
#num_labels=50
# do_lower_case=True
)
example_text = 'I will watch Memento tonight'
bert_input = tokenizer(example_text,padding='max_length', max_length = 10, 
                       truncation=True, return_tensors="pt")


print(bert_input['input_ids'])
print(bert_input['token_type_ids'])
print(bert_input['attention_mask'])

In [ ]:
import torch
import numpy as np
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels[label] for label in df['class']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])
       # return torch.LongTensor (self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [ ]:
## Раздели данные текста на тестовую и тренировочную части

np.random.seed(112)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42), 
                                     [int(.8*len(df)), int(.9*len(df))])

Создадим 2 переменные, первая переменная, которую мы назвали _ в приведенном ниже коде, содержит векторы встраивания всех токенов в последовательности. Вторая переменная, которую мы назвали pooled_output, содержит вектор встраивания токена [CLS] Затем мы передаем pooled_output переменную в линейный слой с функцией активации ReLU. В конце линейного слоя у нас есть вектор размера 50, каждый из которых соответствует категории наших классов

In [ ]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 50)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

Тренировочный цикл

In [ ]:
from torch.optim import Adam
from tqdm import tqdm

def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')
                  
EPOCHS = 5
model = BertClassifier()
LR = 1e-6
              
train(model, df_train, df_val, LR, EPOCHS)